### Create a nested P2WPKH address

In [9]:
from bitcoinutils.setup import setup
from bitcoinutils.utils import to_satoshis
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.keys import P2pkhAddress, P2wpkhAddress, P2wshAddress, P2shAddress, PrivateKey, PublicKey
from bitcoinutils.script import Script

In [1]:
def nested_p2wpkh_address():
    # setup the network
    setup('testnet')
    
    # keys 
    priv = PrivateKey('cMahea7zqjxrtgAbB7LSGbcQUr1uX1ojuat9jfLSZyWoMw1uxM2E')
    pub = priv.get_public_key()

    # the p2sh script and the corresponding address
    address = pub.get_segwit_address()
    redeem_script = address.to_script_pub_key()
    
    # generate P2SH(P2WPKH) address
    p2sh_addr = P2shAddress.from_script(redeem_script)
    
    print("Nested p2wpkh address: " + p2sh_addr.to_string())

In [2]:
nested_p2wpkh_address()

Nested p2wpkh address: 2MwA22fBcwZf9dTsqAzEXr4cb1WVVQSroo3


### Send to nested P2WPKH address

In [20]:
def send_to_nested_P2WPKH():
    # setup the network
    setup('testnet')


    # create input from tx id of UTXO (contained 0.00093 tBTC)
    txin = TxInput('534bfc7bcef71d78b6eb1ac4bc8dcdf2e8ecfa5802e8be6ea7cb9a59d726e6f2', 0)

    # create tx output using P2WPKH scriptPubKey (locking script)
    addr = P2shAddress('2MwA22fBcwZf9dTsqAzEXr4cb1WVVQSroo3')
    txout = TxOutput(to_satoshis(0.00092), Script(['OP_HASH160', addr.to_hash160(), 'OP_EQUAL']) )
    
    # create transaction from input/output -- default locktime is used
    tx = Transaction([txin], [txout])

    # keys and address corresponding to the UTXO
    sk = PrivateKey('cMahea7zqjxrtgAbB7LSGbcQUr1uX1ojuat9o4KgthY6Fje7Vptx')
    pk = sk.get_public_key().to_hex()
    from_addr = P2pkhAddress('mo6jbdKcdPVsEC8jBDJajspTF4Ymn35h15')
    
    # signature for the input
    sig = sk.sign_input( tx, 0, from_addr.to_script_pub_key() )
    
    # set the scriptSig (unlocking script)
    txin.script_sig = Script([sig, pk])
    signed_tx = tx.serialize()

    # print raw signed transaction ready to be broadcasted
    print("\nRaw signed transaction:\n" + signed_tx)
    print("\nTxId:", tx.get_txid())

In [17]:
send_to_nested_P2WPKH()


Raw signed transaction:
0200000001f2e626d7599acba76ebee80258faece8f2cd8dbcc41aebb6781df7ce7bfc4b53000000006b483045022100e81cb6a4e156b6dbcf9acdca2dfb9e46f819e68a0ac36e2958295921eb972fff022027bc47c4cab251403cd0ca98d1933dca8d9f2950a0471740c5279f454e231e6d0121025ec5f8f5efa0ce5313c80f835c026301f520611fe00cd2491c4eed8beea25c89ffffffff01606701000000000017a9142ae470bf0ebed5676703b8e9c12169c9be78791d8700000000

TxId: cfa38a0c7bf84007059fa7f89a53e00c3f3f656673e32fe8040a16771eec7b49


### Spend nested P2WPKH output

In [18]:
def spend_nested_p2wpkh():
    # setup the network
    setup('testnet')

    # the keys corresponding to the P2WPKH address
    priv = PrivateKey('cMahea7zqjxrtgAbB7LSGbcQUr1uX1ojuat9jfLSZyWoMw1uxM2E')
    pub = priv.get_public_key()

    # the p2sh script and the corresponding address
    redeem_script = pub.get_segwit_address().to_script_pub_key()
    p2sh_addr = P2shAddress.from_script(redeem_script)

    # the UTXO of the P2SH-P2WPKH that we are trying to spend
    inp = TxInput('cfa38a0c7bf84007059fa7f89a53e00c3f3f656673e32fe8040a16771eec7b49', 0)

    # exact amount of UTXO we try to spent
    amount = to_satoshis(0.00092)

    # the address to send funds to
    to_addr = P2pkhAddress('mgkswn7UohYskb8bED9BiQXMC5gMCjSoG3')

    # the output sending 0.0009 - 0.00002 is tx fee - no change
    out = TxOutput(to_satoshis(0.0009), to_addr.to_script_pub_key())

    # create a tx with one segwit input
    tx = Transaction([inp], [out], has_segwit=True)

    # script code corresponding to P2WPKH (it is the classic P2PKH)
    script_code = pub.get_address().to_script_pub_key()
    
    # signature using the appropriate script code and the original amount of the UTXO
    sig = priv.sign_segwit_input(tx, 0, script_code, amount)

    # script_sig is the redeem script passed as a single element
    inp.script_sig = Script([redeem_script.to_hex()])

    # finally, the unlocking script is added as a witness
    tx.witnesses.append(Script([sig, pub.to_hex()]))

    # print raw signed transaction ready to be broadcasted
    print("\nRaw signed transaction:\n" + tx.serialize())
    print("\nTxId:", tx.get_txid())

In [19]:
spend_nested_p2wpkh()


Raw signed transaction:
02000000000101497bec1e77160a04e82fe37366653f3f0ce0539af8a79f050740f87b0c8aa3cf0000000017160014493159593872c5dea28335a7b51db4f45913f405ffffffff01905f0100000000001976a9140d9a5183060f471ad234aaf6b16b46fd7cdb45d488ac0247304402206a2fa3abe05d23c05127ec0287fb397e87146bb69433637f3a8561db9155bf7802200a740538456a923fe1b90221934f8723f955dcb53181706d7b77a6a82e87faca012103ff4ed02d37f55c0d6f420ba513d379dc7c88326b466de43eabbdf0afda3de87700000000

TxId: 9afc697a85109ffb4d4aba57e60e0f6cac5ab3025be4cbc72cb6aab3a9a899f9
